ipl sql

In [1]:
import sqlite3

conn = sqlite3.connect("ipl.db")

In [2]:
!pip install ipython-sql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.9 MB/s eta 0:00:00


In [3]:
%load_ext sql
%sql sqlite:///ipl.db

In [4]:
from google.colab import files
uploaded=files.upload()

Saving Team.csv to Team.csv
Saving Season.csv to Season.csv
Saving Player_Match.csv to Player_Match.csv
Saving Player.csv to Player.csv
Saving Match.csv to Match.csv
Saving Ball_by_Ball.csv to Ball_by_Ball.csv


In [5]:
import pandas as pd

In [6]:
player_df=pd.read_csv('Player.csv')
team_df=pd.read_csv('Team.csv')
season_df=pd.read_csv('Season.csv')
playermatch_df=pd.read_csv('Player_Match.csv')
match_df=pd.read_csv('Match.csv')
ballbyball_df=pd.read_csv('Ball_by_Ball.csv')

In [7]:
player_df.head()

,Player_Id,Player_Name,DOB,Batting_Hand,Bowling_Skill,Country,Is_Umpire,Unnamed: 7
0,1,SC Ganguly,8-Jul-72,Left_Hand,Right-arm medium,India,0,NaN
1,2,BB McCullum,27-Sep-81,Right_Hand,Right-arm medium,New Zealand,0,NaN
2,3,RT Ponting,19-Dec-74,Right_Hand,Right-arm medium,Australia,0,NaN
3,4,DJ Hussey,15-Jul-77,Right_Hand,Right-arm offbreak,Australia,0,NaN
4,5,Mohammad Hafeez,17-Oct-80,Right_Hand,Right-arm offbreak,Pakistan,0,NaN


In [8]:
player_df = player_df.loc[:, ~player_df.columns.str.contains('^Unnamed')]
player_df.head()

,Player_Id,Player_Name,DOB,Batting_Hand,Bowling_Skill,Country,Is_Umpire
0,1,SC Ganguly,8-Jul-72,Left_Hand,Right-arm medium,India,0
1,2,BB McCullum,27-Sep-81,Right_Hand,Right-arm medium,New Zealand,0
2,3,RT Ponting,19-Dec-74,Right_Hand,Right-arm medium,Australia,0
3,4,DJ Hussey,15-Jul-77,Right_Hand,Right-arm offbreak,Australia,0
4,5,Mohammad Hafeez,17-Oct-80,Right_Hand,Right-arm offbreak,Pakistan,0


In [9]:
print(player_df.columns.tolist())
print(team_df.columns.tolist())
print(season_df.columns.tolist())
print(playermatch_df.columns.tolist())
print(match_df.columns.tolist())
print(ballbyball_df.columns.tolist())

['Player_Id', 'Player_Name', 'DOB', 'Batting_Hand', 'Bowling_Skill', 'Country', 'Is_Umpire']
['Team_Id', 'Team_Name', 'Team_Short_Code']
['Season_Id', 'Season_Year', 'Orange_Cap_Id', 'Purple_Cap_Id', 'Man_of_the_Series_Id']
['Match_Id', 'Player_Id', 'Team_Id', 'Is_Keeper', 'Is_Captain']
['Match_Id', 'Match_Date', 'Team_Name_Id', 'Opponent_Team_Id', 'Season_Id', 'Venue_Name', 'Toss_Winner_Id', 'Toss_Decision', 'IS_Superover', 'IS_Result', 'Is_DuckWorthLewis', 'Win_Type', 'Won_By', 'Match_Winner_Id', 'Man_Of_The_Match_Id', 'First_Umpire_Id', 'Second_Umpire_Id', 'City_Name', 'Host_Country']
['Match_Id', 'Innings_Id', 'Over_Id', 'Ball_Id', 'Team_Batting_Id', 'Team_Bowling_Id', 'Striker_Id', 'Striker_Batting_Position', 'Non_Striker_Id', 'Bowler_Id', 'Batsman_Scored', 'Extra_Type', 'Extra_Runs', 'Player_dissimal_Id', 'Dissimal_Type', 'Fielder_Id']


In [10]:
team_df.dtypes

,0
Team_Id,int64
Team_Name,object
Team_Short_Code,object


In [11]:
%%sql
CREATE TABLE Player (
    Player_Id INTEGER PRIMARY KEY,
    Player_Name TEXT NOT NULL,
    DOB DATE,
    Batting_Hand TEXT,
    Bowling_Skill TEXT,
    Country TEXT,
    Is_Umpire INTEGER CHECK (Is_Umpire IN (0, 1))
);

CREATE TABLE Team (
    Team_Id INTEGER PRIMARY KEY,
    Team_Name TEXT NOT NULL,
    Team_Short_Code TEXT
);

CREATE TABLE Season (
    Season_Id INTEGER PRIMARY KEY,
    Season_Year INTEGER,
    Orange_Cap_Id INTEGER,
    Purple_Cap_Id INTEGER,
    Man_of_the_Series_Id INTEGER,
    FOREIGN KEY (Orange_Cap_Id) REFERENCES Player(Player_Id),
    FOREIGN KEY (Purple_Cap_Id) REFERENCES Player(Player_Id),
    FOREIGN KEY (Man_of_the_Series_Id) REFERENCES Player(Player_Id)
);

CREATE TABLE Match (
    Match_Id INTEGER PRIMARY KEY,
    Match_Date DATE,
    Team_Name_Id INTEGER,
    Opponent_Team_Id INTEGER,
    Season_Id INTEGER,
    Venue_Name TEXT,
    Toss_Winner_Id INTEGER,
    Toss_Decision TEXT,
    IS_Superover INTEGER CHECK (IS_Superover IN (0, 1)),
    IS_Result INTEGER CHECK (IS_Result IN (0, 1)),
    Is_DuckWorthLewis INTEGER CHECK (Is_DuckWorthLewis IN (0, 1)),
    Win_Type TEXT,
    Won_By REAL,
    Match_Winner_Id INTEGER,
    Man_Of_The_Match_Id INTEGER,
    First_Umpire_Id INTEGER,
    Second_Umpire_Id INTEGER,
    City_Name TEXT,
    Host_Country TEXT,
    FOREIGN KEY (Team_Name_Id) REFERENCES Team(Team_Id),
    FOREIGN KEY (Opponent_Team_Id) REFERENCES Team(Team_Id),
    FOREIGN KEY (Season_Id) REFERENCES Season(Season_Id),
    FOREIGN KEY (Toss_Winner_Id) REFERENCES Team(Team_Id),
    FOREIGN KEY (Match_Winner_Id) REFERENCES Team(Team_Id),
    FOREIGN KEY (Man_Of_The_Match_Id) REFERENCES Player(Player_Id),
    FOREIGN KEY (First_Umpire_Id) REFERENCES Player(Player_Id),
    FOREIGN KEY (Second_Umpire_Id) REFERENCES Player(Player_Id)
);

CREATE TABLE Player_Match (
    Match_Id INTEGER,
    Player_Id INTEGER,
    Team_Id INTEGER,
    Is_Keeper INTEGER CHECK (Is_Keeper IN (0, 1)),
    Is_Captain INTEGER CHECK (Is_Captain IN (0, 1)),
    PRIMARY KEY (Match_Id, Player_Id),
    FOREIGN KEY (Match_Id) REFERENCES Match(Match_Id),
    FOREIGN KEY (Player_Id) REFERENCES Player(Player_Id),
    FOREIGN KEY (Team_Id) REFERENCES Team(Team_Id)
);

CREATE TABLE Ball_by_Ball (
    Match_Id INTEGER,
    Innings_Id INTEGER,
    Over_Id INTEGER,
    Ball_Id INTEGER,
    Team_Batting_Id INTEGER,
    Team_Bowling_Id INTEGER,
    Striker_Id INTEGER,
    Striker_Batting_Position INTEGER,
    Non_Striker_Id INTEGER,
    Bowler_Id INTEGER,
    Batsman_Scored TEXT,
    Extra_Type TEXT,
    Extra_Runs INTEGER,
    Player_dissimal_Id INTEGER,
    Dissimal_Type TEXT,
    Fielder_Id INTEGER,
    PRIMARY KEY (Match_Id, Innings_Id, Over_Id, Ball_Id),
    FOREIGN KEY (Match_Id) REFERENCES Match(Match_Id),
    FOREIGN KEY (Team_Batting_Id) REFERENCES Team(Team_Id),
    FOREIGN KEY (Team_Bowling_Id) REFERENCES Team(Team_Id),
    FOREIGN KEY (Striker_Id) REFERENCES Player(Player_Id),
    FOREIGN KEY (Non_Striker_Id) REFERENCES Player(Player_Id),
    FOREIGN KEY (Bowler_Id) REFERENCES Player(Player_Id),
    FOREIGN KEY (Player_dissimal_Id) REFERENCES Player(Player_Id),
    FOREIGN KEY (Fielder_Id) REFERENCES Player(Player_Id)
);

 * sqlite:///ipl.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [12]:
player_df.to_sql('Player', conn, if_exists='replace', index=False)
team_df.to_sql('Team', conn, if_exists='replace', index=False)
season_df.to_sql('Season', conn, if_exists='replace', index=False)
match_df.to_sql('Match', conn, if_exists='replace', index=False)
playermatch_df.to_sql('Player_Match', conn, if_exists='replace', index=False)
ballbyball_df.to_sql('Ball_by_Ball', conn, if_exists='replace', index=False)

136590

In [13]:
import pandas as pd

query = "SELECT * FROM Player LIMIT 5"
df = pd.read_sql(query, conn)
df.head()

,Player_Id,Player_Name,DOB,Batting_Hand,Bowling_Skill,Country,Is_Umpire
0,1,SC Ganguly,8-Jul-72,Left_Hand,Right-arm medium,India,0
1,2,BB McCullum,27-Sep-81,Right_Hand,Right-arm medium,New Zealand,0
2,3,RT Ponting,19-Dec-74,Right_Hand,Right-arm medium,Australia,0
3,4,DJ Hussey,15-Jul-77,Right_Hand,Right-arm offbreak,Australia,0
4,5,Mohammad Hafeez,17-Oct-80,Right_Hand,Right-arm offbreak,Pakistan,0


In [14]:
query = "SELECT * FROM Ball_by_Ball"
df = pd.read_sql(query, conn)
df.head()

,Match_Id,Innings_Id,Over_Id,Ball_Id,Team_Batting_Id,Team_Bowling_Id,Striker_Id,Striker_Batting_Position,Non_Striker_Id,Bowler_Id,Batsman_Scored,Extra_Type,Extra_Runs,Player_dissimal_Id,Dissimal_Type,Fielder_Id
0,335987,1,1,1,1,2,1,1,2,14,0,legbyes,1,,,
1,335987,1,1,2,1,2,2,2,1,14,0,,,,,
2,335987,1,1,3,1,2,2,2,1,14,0,wides,1,,,
3,335987,1,1,4,1,2,2,2,1,14,0,,,,,
4,335987,1,1,5,1,2,2,2,1,14,0,,,,,


In [15]:
#Top 10 Run Scorers
query= '''SELECT
    p.Player_Name,
    SUM(b.Batsman_Scored) AS Total_Runs
FROM
    Ball_by_Ball b
JOIN
    Player p ON b.Striker_Id = p.Player_Id
GROUP BY
    b.Striker_Id
ORDER BY
    Total_Runs DESC
LIMIT 10; '''
df=pd.read_sql(query,conn)
df

,Player_Name,Total_Runs
0,SK Raina,4106.0
1,V Kohli,4105.0
2,RG Sharma,3874.0
3,G Gambhir,3634.0
4,CH Gayle,3447.0
5,RV Uthappa,3390.0
6,DA Warner,3373.0
7,AB de Villiers,3270.0
8,MS Dhoni,3270.0
9,S Dhawan,3082.0


In [16]:
#Top 10 Wicket Takers
query='''SELECT
    p.Player_Name,
    COUNT(*) AS Total_Wickets
FROM
    Ball_by_Ball b
JOIN
    Player p ON b.Bowler_Id = p.Player_Id
WHERE
    b.Dissimal_Type IN ('bowled', 'caught', 'lbw', 'stumped', 'hit wicket')
GROUP BY
    b.Bowler_Id
ORDER BY
    Total_Wickets DESC
LIMIT 10;'''
pd.read_sql(query,conn)

,Player_Name,Total_Wickets
0,SL Malinga,118
1,PP Chawla,98
2,A Mishra,97
3,Harbhajan Singh,97
4,DJ Bravo,90
5,A Nehra,80
6,R Ashwin,76
7,PP Ojha,76
8,Z Khan,74
9,RP Singh,72


In [17]:
#Team with Most Wins
query='''SELECT
    t.team_name,
    t.Team_Short_Code,
    COUNT(*) AS Total_Wins
FROM
    match m
JOIN
    team t ON m.match_winner_Id = t.team_Id
GROUP BY
    m.match_winner_Id
ORDER BY
    Total_Wins DESC
LIMIT 10;'''
pd.read_sql(query,conn)

,Team_Name,Team_Short_Code,Total_Wins
0,Mumbai Indians,MI,80
1,Chennai Super Kings,CSK,79
2,Royal Challengers Bangalore,RCB,70
3,Kolkata Knight Riders,KKR,68
4,Rajasthan Royals,RR,63
5,Kings XI Punjab,KXIP,63
6,Delhi Daredevils,DD,56
7,Sunrisers Hyderabad,SRH,34
8,Deccan Chargers,DC,29
9,Pune Warriors,PW,12


In [18]:
#Player with Most Dismissals as Fielder
query='''SELECT
    p.Player_Name,
    COUNT(*) AS Total_Dismissals
FROM
    Ball_by_Ball b
JOIN
    Player p ON b.Fielder_Id = p.Player_Id
WHERE
    b.Dissimal_Type IN ('caught', 'run out')
GROUP BY
    b.Fielder_Id
ORDER BY
    Total_Dismissals DESC
LIMIT 10;
'''
pd.read_sql(query,conn)

,Player_Name,Total_Dismissals
0,KD Karthik,84
1,AB de Villiers,83
2,SK Raina,78
3,MS Dhoni,66
4,RV Uthappa,65
5,V Kohli,56
6,RG Sharma,55
7,PA Patel,54
8,NV Ojha,51
9,KA Pollard,48


In [19]:
#Players Who Were Also Umpires
query='''select p.player_name from player p where is_umpire=1'''
pd.read_sql(query,conn)

,Player_Name
0,Asad Rauf
1,MR Benson
2,Aleem Dar
3,SJ Davis
4,BF Bowden
5,IL Howell
6,DJ Harper
7,RE Koertzen
8,BR Doctrove
9,AV Jayaprakash


In [20]:
#Super Overs per Season
query='''SELECT
    s.Season_Year,
    COUNT(*) AS Total_Superovers_Per_Year
FROM
    Season s
JOIN
    Match m ON m.Season_Id = s.Season_Id
WHERE
    m.Is_SuperOver = 1
GROUP BY
    s.Season_Year;'''
pd.read_sql(query,conn)

,Season_Year,Total_Superovers_Per_Year
0,2009,1
1,2010,1
2,2013,2
3,2014,1
4,2015,1


In [21]:
#Most Matches Played by a Player
query='''SELECT
    p.Player_Name,
    COUNT(pm.Match_Id) AS Matches_Played
FROM
    Player_Match pm
JOIN
    Player p ON pm.Player_Id = p.Player_Id
GROUP BY
    pm.Player_Id
ORDER BY
    Matches_Played DESC
LIMIT 10;'''
pd.read_sql(query,conn)

,Player_Name,Matches_Played
0,SK Raina,146
1,RG Sharma,142
2,MS Dhoni,142
3,KD Karthik,138
4,V Kohli,138
5,RV Uthappa,135
6,YK Pathan,134
7,G Gambhir,132
8,Harbhajan Singh,125
9,RA Jadeja,125


In [22]:
#Bowler ID
query=''' select p.player_id, p.player_name from player p where p.player_name like '%Sharma%' '''
pd.read_sql(query,conn)

,Player_Id,Player_Name
0,57,RG Sharma
1,84,I Sharma
2,119,Joginder Sharma
3,166,KV Sharma
4,228,Bipul Sharma
5,235,R Sharma
6,314,Ankit Sharma
7,364,MM Sharma
8,367,Sandeep Sharma
9,381,Shivam Sharma


In [23]:
#Players Dismissed Most by a Specific Bowler
query = '''SELECT
    dismissed.Player_Name AS Batsman_Dismissed,
    COUNT(*) AS Times_Dismissed
FROM
    Ball_by_Ball b
JOIN
    Player dismissed ON b.Player_dissimal_Id = dismissed.Player_Id
WHERE
    b.Bowler_Id = 57
    AND b.Dissimal_Type IN ('bowled', 'lbw', 'caught', 'stumped', 'hit wicket')
GROUP BY
    b.Player_dissimal_Id
ORDER BY
    Times_Dismissed DESC
LIMIT 10;'''
pd.read_sql(query,conn)

,Batsman_Dismissed,Times_Dismissed
0,SS Tiwary,1
1,S Sohal,1
2,Harbhajan Singh,1
3,AM Nayar,1
4,SK Warne,1
5,SM Katich,1
6,KC Sangakkara,1
7,R Ashwin,1
8,M Morkel,1
9,M Vijay,1


In [24]:
#Which City Hosted the Most Matches?
query='''SELECT
    m.Venue_Name, m.city_name, m.host_country,
    COUNT(m.Match_Id) AS Number_Of_Matches_Held
FROM
    Match m
GROUP BY
    m.Venue_Name
ORDER BY
    Number_Of_Matches_Held DESC;
'''
pd.read_sql(query,conn)

,Venue_Name,City_Name,Host_Country,Number_Of_Matches_Held
0,M Chinnaswamy Stadium,Bangalore,India,58
1,Eden Gardens,Kolkata,India,54
2,Feroz Shah Kotla,Delhi,India,53
3,Wankhede Stadium,Mumbai,India,49
4,"MA Chidambaram Stadium, Chepauk",Chennai,India,48
5,"Rajiv Gandhi International Stadium, Uppal",Hyderabad,India,41
6,"Punjab Cricket Association Stadium, Mohali",Chandigarh,India,35
7,Sawai Mansingh Stadium,Jaipur,India,33
8,Subrata Roy Sahara Stadium,Pune,India,17
9,Dr DY Patil Sports Academy,Mumbai,India,17


In [25]:
#Most Common Dismissal Types
query='''
SELECT
    Dissimal_Type,
    COUNT(*) AS Total_Dismissals
FROM
    Ball_by_Ball
WHERE
    Dissimal_Type IS NOT NULL
GROUP BY
    Dissimal_Type
ORDER BY
    Total_Dismissals DESC;
'''
pd.read_sql(query,conn)

,Dissimal_Type,Total_Dismissals
0,,131058
1,caught,3245
2,bowled,1028
3,run out,583
4,lbw,326
5,stumped,187
6,caught and bowled,148
7,retired hurt,8
8,hit wicket,6
9,obstructing the field,1


In [26]:
query='select * from ball_by_ball'
pd.read_sql(query,conn)

,Match_Id,Innings_Id,Over_Id,Ball_Id,Team_Batting_Id,Team_Bowling_Id,Striker_Id,Striker_Batting_Position,Non_Striker_Id,Bowler_Id,Batsman_Scored,Extra_Type,Extra_Runs,Player_dissimal_Id,Dissimal_Type,Fielder_Id
0,335987,1,1,1,1,2,1,1,2,14,0,legbyes,1,,,
1,335987,1,1,2,1,2,2,2,1,14,0,,,,,
2,335987,1,1,3,1,2,2,2,1,14,0,wides,1,,,
3,335987,1,1,4,1,2,2,2,1,14,0,,,,,
4,335987,1,1,5,1,2,2,2,1,14,0,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136585,981024,2,20,2,2,11,369,6,434,299,2,,,,,
136586,981024,2,20,3,2,11,369,6,434,299,0,,,434,run out,183
136587,981024,2,20,4,2,11,140,9,369,299,0,legbyes,1,,,
136588,981024,2,20,5,2,11,369,6,140,299,1,,,,,


In [27]:
#strike rate
query='''
SELECT
    p.Player_Name,
    stats.Total_Runs,
    stats.Balls_Faced,
    ROUND((stats.Total_Runs * 100.0) / stats.Balls_Faced, 2) AS Strike_Rate
FROM (
    SELECT
        b.Striker_Id,
        SUM(b.Batsman_Scored) AS Total_Runs,
        COUNT(*) AS Balls_Faced
    FROM
        Ball_by_Ball b
    WHERE
        b.Extra_Type IS NULL OR b.Extra_Type NOT IN ('wides')
    GROUP BY
        b.Striker_Id
) AS stats
JOIN
    Player p ON stats.Striker_Id = p.Player_Id
WHERE
    stats.Balls_Faced >= 1000
ORDER BY
    Strike_Rate DESC
LIMIT 10;

'''
pd.read_sql(query,conn)

,Player_Name,Total_Runs,Balls_Faced,Strike_Rate
0,V Sehwag,2728.0,1768,154.30
1,CH Gayle,3447.0,2277,151.38
2,AB de Villiers,3270.0,2191,149.25
3,KA Pollard,1959.0,1331,147.18
4,YK Pathan,2779.0,1898,146.42
5,DA Miller,1480.0,1028,143.97
6,DA Warner,3373.0,2384,141.48
7,SR Watson,2553.0,1823,140.04
8,SK Raina,4106.0,2967,138.39
9,MS Dhoni,3270.0,2366,138.21
